In [1]:
import requests
import json
import random

BASE_URL = "http://localhost:8000"

# IMPORTANT: Set this to a real user UUID from /auth/login
USER_ID = "4ac4a58c-31a8-46f0-84b4-e245537237bd"  # e.g. "6e0e5c7e-1234-4567-890a-bcdef1234567"

if USER_ID == "PUT-YOUR-USER-ID-HERE":
    print("⚠️ Please set USER_ID to a real UUID from /auth/login before running interactions tests.")
else:
    print("USER_ID set to:", USER_ID)

HEADERS = {"X-User-ID": USER_ID} if USER_ID else {}
print("BASE_URL =", BASE_URL)


USER_ID set to: 4ac4a58c-31a8-46f0-84b4-e245537237bd
BASE_URL = http://localhost:8000


## 1️⃣ Ensure a Test Location Exists

Interactions require a valid `location_id`. We will:
- Try to fetch existing locations.
- If none exist, create a new test location.


In [2]:
def get_any_location_id():
    """Return an existing location_id if available, else None."""
    url = f"{BASE_URL}/locations/"
    try:
        res = requests.get(url)
        if res.status_code != 200:
            print("GET /locations failed with status:", res.status_code)
            return None
        locations = res.json()
        if not locations:
            return None
        return locations[0]["id"]
    except Exception as e:
        print("Error fetching locations:", e)
        return None

test_location_id = get_any_location_id()
print("Existing test_location_id:", test_location_id)

if not test_location_id:
    # Create a fallback test location
    create_payload = {
        "name": "Interactions Test Location",
        "description": "Location created for interactions API tests.",
        "maps_url": "https://maps.example.com/interactions-test",
        "price_level": random.choice([1, 2, 3, 4]),
        "area": "Test Area",
    }
    url = f"{BASE_URL}/locations/"
    res = requests.post(url, json=create_payload)
    print("Created fallback test location → Status", res.status_code)
    data = res.json()
    test_location_id = data.get("id")

print("Using test_location_id:", test_location_id)
test_location_id


Existing test_location_id: de3763f6-41a4-4370-bb03-0baa7f921ecc
Using test_location_id: de3763f6-41a4-4370-bb03-0baa7f921ecc


'de3763f6-41a4-4370-bb03-0baa7f921ecc'

## 2️⃣ Like a Location – `POST /interactions/like/{location_id}`

We record a **like** for the current user on the test location.


In [3]:
if not USER_ID or USER_ID == "PUT-YOUR-USER-ID-HERE":
    raise RuntimeError("USER_ID is not set. Please set it in the first cell.")

if not test_location_id:
    raise RuntimeError("No test_location_id available. Check the previous cell.")

url = f"{BASE_URL}/interactions/like/{test_location_id}"
res = requests.post(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "message": "Location liked"
}


## 3️⃣ Get User Likes – `GET /interactions/likes`

Verify that the like is visible in the user's likes list.


In [4]:
url = f"{BASE_URL}/interactions/likes"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
likes = res.json()
print("Total likes:", len(likes))
print(json.dumps(likes[:5], indent=2))


Status: 200
Total likes: 1
[
  {
    "user_id": "4ac4a58c-31a8-46f0-84b4-e245537237bd",
    "location_id": "de3763f6-41a4-4370-bb03-0baa7f921ecc",
    "created_at": "2025-11-23T15:02:01.622646+08:00"
  }
]


## 4️⃣ Unlike a Location – `DELETE /interactions/like/{location_id}`

Remove the like for the test location.


In [5]:
url = f"{BASE_URL}/interactions/like/{test_location_id}"
res = requests.delete(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "message": "Like removed"
}


## 5️⃣ Verify Likes After Unlike – `GET /interactions/likes`

Ensure the test location is no longer in the likes list.


In [6]:
url = f"{BASE_URL}/interactions/likes"
res = requests.get(url, headers=HEADERS)
print("Status:", res.status_code)
likes_after = res.json()
print("Total likes now:", len(likes_after))
print(json.dumps(likes_after[:5], indent=2))


Status: 200
Total likes now: 0
[]


## 6️⃣ Save a Location – `POST /interactions/save/{location_id}`

Record a **save** action for the test location.


In [7]:
url = f"{BASE_URL}/interactions/save/{test_location_id}"
res = requests.post(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "message": "Location saved"
}


## 7️⃣ Get Saved Locations – `GET /interactions/saved`

Verify that the save appears in the saved list for the user.


In [8]:
url = f"{BASE_URL}/interactions/saved"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
saved = res.json()
print("Total saved locations:", len(saved))
print(json.dumps(saved[:5], indent=2))


Status: 200
Total saved locations: 1
[
  {
    "user_id": "4ac4a58c-31a8-46f0-84b4-e245537237bd",
    "location_id": "de3763f6-41a4-4370-bb03-0baa7f921ecc",
    "created_at": "2025-11-23T15:02:18.224999+08:00"
  }
]


## 8️⃣ Unsave a Location – `DELETE /interactions/save/{location_id}`

Remove the save for the test location.


In [9]:
url = f"{BASE_URL}/interactions/save/{test_location_id}"
res = requests.delete(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "message": "Removed from saved"
}


## 9️⃣ Verify Saved List After Unsave – `GET /interactions/saved`

Ensure the test location is no longer in the saved list.


In [10]:
url = f"{BASE_URL}/interactions/saved"
res = requests.get(url, headers=HEADERS)
print("Status:", res.status_code)
saved_after = res.json()
print("Total saved locations now:", len(saved_after))
print(json.dumps(saved_after[:5], indent=2))


Status: 200
Total saved locations now: 0
[]


## 🔟 Record a Visit – `POST /interactions/visit/{location_id}`

Record a **visit** event for the user at the test location.


In [11]:
url = f"{BASE_URL}/interactions/visit/{test_location_id}"
res = requests.post(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## 1️⃣1️⃣ Get Visit History – `GET /interactions/visits`

Verify that the visit appears in the user's visit history.


In [12]:
url = f"{BASE_URL}/interactions/visits"
res = requests.get(url, headers=HEADERS)
print("Status:", res.status_code)
visits = res.json()
print("Total visits:", len(visits))
print(json.dumps(visits[:5], indent=2))


Status: 200
Total visits: 0
[]


## ✅ Done!

You have now tested all key interactions endpoints for a single user:
- Likes: create, list, delete
- Saved locations: create, list, delete
- Visits: record and list

You can repeat these tests with different users by changing `USER_ID` in the first cell.
